<a href="https://colab.research.google.com/github/Kambojharsh/Tweet-Support-Classifier/blob/main/BTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=6df91ef02d559f09d24266d1d18aa7798e0439ad6c81b07c2ea7b5e61b7f6ba4
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [ ]:
import pandas as pd
import random as rd
import re
import emoji
import math
import string
import numpy as np
import wordninja
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df= pd.read_csv('/content/drive/MyDrive/DL/annotations - Final_Dataset.csv')
df = df[df['Reply_Class'] != 'Lang']

# Preprocessing


In [ ]:

def pre_process_tweet(tweet):
    # Remove \n from the end of the sentence
    tweet = tweet.strip('\n')

    # Convert emojis to their descriptions
    tweet = emoji.demojize(tweet)
    # Add spaces between emoji descriptions
    tweet = re.sub(r'(:\w+:)', lambda x: x.group(0).replace(':', ' '), tweet)

    # Remove any URL
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = re.sub(r"www\S+", "", tweet)

    # Convert underscore-separated words to space-separated words
    tweet = re.sub(r'_', ' ', tweet)

    # Convert dash-separated words to space-separated words
    tweet = re.sub(r'-', ' ', tweet)

    # Replace &amp with 'and'
    tweet = tweet.replace('&amp','and')
    tweet = tweet.replace('&AMP','and')

    # Replace U.S. with 'usa'
    tweet = tweet.replace('U.S.', 'usa')
    tweet = tweet.replace('US','usa')
    # Remove colons from the end of the sentences (if any)
    tweet = tweet.strip()
    if tweet.endswith(':'):
        tweet = tweet[:-1]

    # Split tweet into words
    # words = tweet.split()

    # # Iterate over words and replace hashtag words
    # for i, word in enumerate(words):
    #     if word.startswith('#'):
    #         words[i] = separate_hashtag_words(word)

    # tweet = ' '.join(words)
    # tweet = re.sub(r'#\w+', lambda x: separate_hashtag_words(x.group()), tweet)
    # Remove hash-tags symbols and add spaces between words
    # tweet = re.sub(r'#(\w+)', r' \1', tweet)

    # Convert every word to lowercase
    tweet = tweet.lower()


    tweet = tweet.replace('#', '')
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Trim extra spaces
    tweet = " ".join(tweet.split())

    return tweet

# Example usage:
tweet = "#sanctionpakistan"
processed_tweet = pre_process_tweet(tweet)
print(processed_tweet)
tweet = "😙😚Here is an 'example' tweet! @user #example PM's us US  #sanctionpakistan http://example.com  😂💀💀do a lot !!"
processed_tweet = pre_process_tweet(tweet)
print(processed_tweet)



sanctionpakistan
kissing face with smiling eyes kissing face with closed eyes here is an example tweet user example pms us usa sanctionpakistan face with tears of joy skull skull do a lot


In [ ]:
df['Text'] = df['Text'].apply(pre_process_tweet)
df['Reply'] = df['Reply'].apply(pre_process_tweet)

In [ ]:
# Used to seperate words
df['Text'] = df['Text'].apply(lambda x: ' '.join(wordninja.split(x)))
df['Reply'] = df['Reply'].apply(lambda x: ' '.join(wordninja.split(x)))

In [ ]:
print(df['Reply'][11])

sanction pakistan united nations united nations high commissioner for human rights united nations general assembly international court of justice international courts


# Tokenization and Padding


In [ ]:
# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'] + df['Reply'])

max_sequence_length = 60

sequences_text = tokenizer.texts_to_sequences(df['Text'])
sequences_reply = tokenizer.texts_to_sequences(df['Reply'])

padded_sequences_text = pad_sequences(sequences_text, maxlen=max_sequence_length, padding='post')
padded_sequences_reply = pad_sequences(sequences_reply, maxlen=max_sequence_length, padding='post')



reply_tokens = [tokenizer.index_word.get(token, '') for token in padded_sequences_reply[11]]
print("Reply tokens:", reply_tokens)
print(padded_sequences_reply[11])

Reply tokens: ['sanction', 'pakistan', 'united', 'nations', 'united', 'nations', 'high', 'commissioner', 'for', 'human', 'rights', 'united', 'nations', 'general', 'assembly', 'international', 'court', 'of', 'justice', 'international', 'courts', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
[  54   19  372  794  372  794 1350 4423   11  469  564  372  794  897
 4424  495 2497    4  531  495 3122    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


# Glove dictionary creation

In [ ]:

words = dict()

def add_to_dict(d, filename):
  with open(filename, 'r') as f:
    for line in f.readlines():
      line = line.split(' ')

      try:
        d[line[0]] = np.array(line[1:], dtype=float)
      except:
        continue

add_to_dict(words, '/content/drive/MyDrive/DL/glove.twitter.27B.50d.txt')
words

#Create Embeddings

In [ ]:


# Function to get embeddings for sequences
def get_embeddings(sequences, word_dict):
    embedding_dim = 50
    # embedding_dim = len(next(iter(word_dict.values())))  # Dimension of GloVe embeddings
    num_words = len(tokenizer.word_index) + 1  # Adding 1 for padding token (index 0)

    # Initialize embeddings matrix
    embeddings_matrix = np.zeros((len(sequences), max_sequence_length, embedding_dim))

    # Iterate over sequences
    for i, seq in enumerate(sequences):
        # Iterate over tokens in each sequence
        for j, token_index in enumerate(seq):
            if token_index != 0:  # Skip padding token
                word = tokenizer.index_word.get(token_index)
                if word in word_dict:
                    embeddings_matrix[i, j, :] = word_dict[word]
                else:
                    # If word not in GloVe dictionary, use zero vector
                    embeddings_matrix[i, j, :] = np.zeros(embedding_dim)

    return embeddings_matrix

# Get embeddings for padded_sequences_text and padded_sequences_reply
embeddings_text = get_embeddings(padded_sequences_text, words)
embeddings_reply = get_embeddings(padded_sequences_reply, words)

# Print the shape of embeddings for verification
# print("Shape of embeddings_text:", embeddings_text.shape)
# print("Shape of embeddings_reply:", embeddings_reply.shape)

print(embeddings_reply.shape)
print(embeddings_text.shape)

(2864, 60, 50)
(2864, 60, 50)


# CREATION OF TRAIN, TEST, VAL

In [ ]:
y=df['Reply_Class']

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have your data stored in embeddings_text and embeddings_reply arrays
# Split the data into training (80%), validation (10%), and test (10%) sets
X_train_text, X_temp_text = train_test_split(embeddings_text, test_size=0.2, random_state=42)
X_val_text, X_test_text = train_test_split(X_temp_text, test_size=0.5, random_state=42)

X_train_reply, X_temp_reply = train_test_split(embeddings_reply, test_size=0.2, random_state=42)
X_val_reply, X_test_reply = train_test_split(X_temp_reply, test_size=0.5, random_state=42)

# Assuming you have labels stored in a variable y
# Split the labels accordingly
y_train, y_temp = train_test_split(y, test_size=0.2, random_state=42)
y_val, y_test = train_test_split(y_temp, test_size=0.5, random_state=42)

# Convert the arrays to numpy arrays
X_train_text = np.array(X_train_text)
X_val_text = np.array(X_val_text)
X_test_text = np.array(X_test_text)

X_train_reply = np.array(X_train_reply)
X_val_reply = np.array(X_val_reply)
X_test_reply = np.array(X_test_reply)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Print the shapes to verify
print("X_train_text shape:", X_train_text.shape)
print("X_val_text shape:", X_val_text.shape)
print("X_test_text shape:", X_test_text.shape)
print("X_train_reply shape:", X_train_reply.shape)
print("X_val_reply shape:", X_val_reply.shape)
print("X_test_reply shape:", X_test_reply.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)


#LSTM MODEL

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Flatten, Input

In [ ]:
def build_model(input_shape):
    # Define input layers for text and reply embeddings
    text_input = Input(shape=input_shape)  # Assuming input shape is (60, 50) for text embeddings
    reply_input = Input(shape=input_shape)  # Assuming input shape is (60, 50) for reply embeddings

    # # LSTM layer for text
    lstm_text = LSTM(64, return_sequences=True)(text_input)

    # # LSTM layer for reply
    lstm_reply = LSTM(64, return_sequences=True)(reply_input)

    # # Concatenate the outputs of both LSTM layers
    concatenated = Concatenate(axis=1)([lstm_text, lstm_reply])

    # # # Flatten the concatenated output
    flattened = Flatten()(concatenated)

    # # # Dense layer with softmax activation for 3 classes
    output = Dense(3, activation='softmax')(lstm_text)

    model = Model(inputs=[text_input, reply_input], outputs=output)
    model.summary()
    return model




In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=1, batch_size=32):
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, verbose=1)
    return history

In [ ]:
def evaluate_model(model, X_test, y_test, batch_size=32):
    # Evaluate the model on test data
    test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)

In [ ]:
input_shape = (60, 50)  # Assuming input shape is (60, 50) for text and reply embeddings
model = build_model(input_shape)
# model.summary()
history = train_model(model, X_train_text, y_train, X_val_text, y_val, 1, 32)
evaluate_model(model , X_test_text, y_test, batch_size=32)